In [18]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

In [24]:
column_names = pd.read_csv("hw3-data/my_train.csv", nrows=1).columns.tolist()

train_data = pd.read_csv("hw3-data/my_train.csv", sep=",", skiprows=1, names=column_names, engine="python")
dev_data = pd.read_csv("hw3-data/my_dev.csv", sep=",", skiprows=1, names=column_names, engine="python")
test_data = pd.read_csv("hw3-data/test.csv", sep=",", skiprows=1, names=column_names[:-1], engine="python")

# Separate features and target variable
X_train = train_data[column_names[1:-1]]  # Features
y_train = np.log(train_data['SalePrice'])  # Target variable

X_dev = dev_data[column_names[1:-1]]  # Features
y_dev = np.log(dev_data['SalePrice'])  # Target variable

num_cols = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'OverallQual', 'OverallCond',
    'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
    'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'MoSold', 'YrSold'
]

cat_cols = np.array([c for c in column_names[1:-1] if c not in num_cols])

X_train[num_cols] = X_train[num_cols].fillna(0)
X_dev[num_cols] = X_dev[num_cols].fillna(0)

X_train[cat_cols] = X_train[cat_cols].astype(str)
X_dev[cat_cols] = X_dev[cat_cols].astype(str)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Binarization
X_train_binary = encoder.fit_transform(X_train)
X_dev_binary = encoder.transform(X_dev)

# # Train linear regression model
model = LinearRegression()
model.fit(X_train_binary, y_train)

alphas = np.logspace(-6, 6, 13)

# Train Ridge regression model

# for alpha in alphas:
# # alpha = 5.0  # Regularization strength
#     ridge_model = Ridge(alpha=alpha)
#     ridge_model.fit(X_train_binary, y_train)

# Make predictions
y_pred = model.predict(X_dev_binary)
np.sqrt(mean_squared_error(y_dev, y_pred))

    # y_pred = ridge_model.predict(X_dev_binary)

    # rmsle = np.sqrt(mean_squared_error(y_dev, y_pred))
    # print(f'Alpha: {alpha}, \tRMSLE on Dev Set: {rmsle}')

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_9725/816013159.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[num_cols] = X_train[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_9725/816013159.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dev[num_cols] = X_dev[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_9725/816013159.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

0.15162349569057895

In [13]:
# Get feature names from one-hot encoder
feature_names = encoder.get_feature_names_out()

# Create a DataFrame with feature names and their coefficients
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': model.coef_})

# Sort the DataFrame by coefficients in descending order
sorted_coef_df = coef_df.sort_values(by='Coefficient', ascending=False)

# Top 10 most positive features
top_positive_features = sorted_coef_df.head(10)

# Sort the DataFrame by coefficients in ascending order
sorted_coef_df = coef_df.sort_values(by='Coefficient', ascending=True)

# Top 10 most negative features
top_negative_features = sorted_coef_df.head(10)

# Print the results
print("Top 10 Most Positive Features:")
print(top_positive_features)

print("\nTop 10 Most Negative Features:")
print(top_negative_features)

Top 10 Most Positive Features:
                   Feature  Coefficient
5900            FullBath_3     0.139233
1203         OverallQual_9     0.137390
1162  Neighborhood_StoneBr     0.125355
4712          2ndFlrSF_472     0.113156
1202         OverallQual_8     0.106514
1398      RoofMatl_WdShngl     0.092514
5283        GrLivArea_1192     0.089529
1155  Neighborhood_NoRidge     0.087764
6060          GarageCars_3     0.086167
391           LotArea_8029     0.085358

Top 10 Most Negative Features:
                Feature  Coefficient
15     MSZoning_C (all)    -0.204502
5165      GrLivArea_968    -0.126627
7040  EnclosedPorch_236    -0.123473
1197      OverallQual_3    -0.112828
2443     BsmtFinSF2_311    -0.108804
420        LotArea_8281    -0.107356
1207      OverallCond_3    -0.101119
6058       GarageCars_1    -0.094178
218        LotArea_5000    -0.091047
1195      OverallQual_1    -0.089260


In [14]:
X_test = test_data[column_names[1:-1]]  # Features

X_test[num_cols] = X_test[num_cols].fillna(0)
X_test[cat_cols] = X_test[cat_cols].astype(str)

X_test_binary = encoder.transform(X_test)

# Make predictions
y_test_pred = np.exp(model.predict(X_test_binary))

# Assigning new predicted column to the dataset
test_data['SalePrice'] = y_test_pred

# Create a new DataFrame with 'ID' and 'SalePrice'
result_df = test_data[['Id', 'SalePrice']]

# Save the DataFrame to a CSV file with a header
result_df.to_csv('naive_binarization.csv', index=False)

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_9725/3672169278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[num_cols] = X_test[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_9725/3672169278.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[cat_cols] = X_test[cat_cols].astype(str)


In [25]:
model.intercept_

12.172846747187549

In [16]:
# Total number of features
print("Total Features:")
print(len(feature_names))

Total Features:
7225
